Phase space plots

In [ ]:
for i in range(84):
    if i in ez: 
        plt.plot(pstr_samples_1['x'][-1,:,i], pstr_samples_1['z'][-1,:,i], color='red')
    elif i in pz:
        plt.plot(pstr_samples_1['x'][-1,:,i], pstr_samples_1['z'][-1,:,i], color='orange')
    else:
        plt.plot(pstr_samples_1['x'][-1,:,i], pstr_samples_1['z'][-1,:,i], color='black')

Downsampling **seeg log power**

In [ ]:
%reset -f
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# fit_trgt = np.load('datasets/id001_ac/AC_fit_trgt.npz')['fit_trgt']
# fit_trgt_ds = fit_trgt[0:-1:20,:]
# plt.figure(figsize=(15,5))
# plt.plot(fit_trgt,alpha=0.3);
# plt.xlabel('time')
# plt.ylabel('slp')
# plt.title('Original')
# plt.figure(figsize=(15,5))
# plt.plot(fit_trgt_ds,alpha=0.3);
# plt.xlabel('time')
# plt.ylabel('slp')
# plt.title('Downsampled')

syn_data = np.load('datasets/id001_ac/AC_syn_tvb_ez=59_pz=82-74.npz')
srcSig = syn_data['src_sig'][2200:3800:20,0,:,0] + syn_data['src_sig'][2200:3800:20,3,:,0]
plt.figure(figsize=(15,10))
plt.subplot(211)
plt.plot(srcSig);

Preprocessing retrospective patients data

In [ ]:
%matplotlib inline
import lib.io.stan
import matplotlib.pyplot as plt
import lib.preprocess.envelope
import numpy as np

In [ ]:
data = lib.io.stan.rload('datasets/RetrospectivePatients/id001_bt/vep/data.R')

In [ ]:
data.keys()

In [ ]:
print(data['counts_triu'].shape, data['nn']*(data['nn']-1)/2.0)

sc = data['counts_triu'].reshape((data['nn'],data['nn']))


In [ ]:
np.where(data['is_first'] == 1)[0][1]
# print(data['is_first'])

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(data['seeg'].T);

In [ ]:
szr_onset_idxs = np.where(data['is_first'] == 1)[0]
print(szr_onset_idxs)
seeg = data['seeg'][:,0:szr_onset_idxs[1]].T
seeg = data['seeg'][:,szr_onset_idxs[3]:-1].T
t = lib.preprocess.envelope.bfilt(seeg, 64, 5.0, 'highpass', axis=0)
plt.figure(figsize=(25,5))
plt.plot(t);
t = lib.preprocess.envelope.bfilt(seeg, 64, 20.0, 'lowpass', axis=0)
plt.figure(figsize=(25,5))
plt.plot(t);
t = lib.preprocess.envelope.bfilt(t, 64, 10.0, 'lowpass', axis=0)
plt.figure(figsize=(25,5))
plt.plot(t);
# slp = lib.preprocess.envelope.compute_fitting_target(seeg, 256)
# seeg_mov_avg = lib.preprocess.envelope.mov_avg(seeg, 10)

In [ ]:
ez_clin_hyp = np.loadtxt('datasets/RetrospectivePatients/id001_bt/tvb/ez_hypothesis.destrieux.txt')

In [ ]:
np.where(ez_clin_hyp == 1)[0]+1

In [ ]:
import mne
import json
import matplotlib.pyplot as plt
import numpy as np
import lib.preprocess.envelope
import lib.io.stan

In [ ]:
data_dir = 'datasets/retro/id001_bt'
with open(f'{data_dir}/seeg/fif/BTcrise1appportable_0006.json') as fd:
    meta_data = json.load(fd)
bad_channels = meta_data['bad_channels']
exclude = meta_data['bad_channels'] + meta_data['non_seeg_channels']
# fif_fname = os.path.join(os.path.dirname(js['_source']), js['filename'])
raw = mne.io.Raw(f'{data_dir}/seeg/fif/BTcrise1appportable_0006.raw.fif', verbose='WARNING')
# raw = mne.io.Raw(fif_fname, verbose='WARNING')
picks = set(raw.ch_names) - set(exclude)
assert meta_data['onset'] is not None and meta_data['termination'] is not None
# raw.crop(tmin=meta_data['onset'], tmax=meta_data['termination'])
raw.load_data()
raw.pick_channels(picks)

In [ ]:
meta_data['onset'], raw.info['sfreq']

In [ ]:
seeg = raw.get_data().T#[0:-1:8,:]
# help(raw.crop)
seeg.shape

In [ ]:
plt.figure(figsize=(25,30))
plt.plot(seeg/np.max(seeg, axis=0) + 2 * np.r_[0:raw.info['nchan']]);
# plt.plot(seeg/np.max(seeg,axis=0),'k');

In [ ]:
# t = np.zeros_like(seeg)

for i in range(seeg.shape[1]):
    ts = seeg[:,i]
    ts[abs(ts - ts.mean())>2*ts.std()] = ts.mean()
#     seeg[:,i] = ts

In [ ]:
plt.figure(figsize=[25,5])
plt.plot(seeg[0:-1:int(raw.info['sfreq']/4)]);

In [ ]:
seeg = lib.preprocess.envelope.bfilt(seeg, raw.info['sfreq'], 10.0, 'highpass', axis=0)
plt.figure(figsize=(25,5))
plt.plot(seeg[0:-1:int(raw.info['sfreq']/4)],'k',alpha=0.3);
plt.title('Highpass at fcut=5')

In [ ]:
seeg = lib.preprocess.envelope.seeg_log_power(seeg, 50)#[0:-65,:]
plt.figure(figsize=(25,5))
plt.plot(seeg[0:-1:int(raw.info['sfreq']/4)],'k',alpha=0.3);
plt.title('log of moving average')

In [ ]:
# t3 = np.zeros_like(t2)

for i in range(seeg.shape[1]):
    ts = seeg[:,i]
    ts[abs(ts - ts.mean())>2*ts.std()] = ts.mean()
#     seeg[:,i] = ts

In [ ]:
plt.figure(figsize=[25,5])
plt.plot(seeg,'k', alpha=0.3);
# plt.plot(t2);

In [ ]:
seeg = lib.preprocess.envelope.bfilt(seeg, raw.info['sfreq'], 0.04, 'lowpass', axis=0)
plt.figure(figsize=(25,5))
plt.plot(seeg[0:-1:int(raw.info['sfreq'])], 'k', alpha=0.3);
plt.title('low pass filtered at 10Hz to smooth')

In [ ]:
slp = seeg[0:-1:int(raw.info['sfreq']),:]
plt.figure(figsize=(25,5))
plt.plot(slp,'k',alpha=0.3)
# plt.vlines(180,3,15,colors='red')
plt.title('Downsampled')

In [ ]:
raw.info['sfreq']

In [ ]:
slp = lib.preprocess.envelope.compute_fitting_target(t,64)
plt.figure(figsize=(25,5))
plt.plot(slp[0:-65:128,:],'k',alpha=0.3);

In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
mode = 10.0
sigma = 0.5
t_star = np.random.normal(0,1,1000)
print(np.mean(sigma*t_star))
# t_star = t_star[np.logical_and(t_star>-1,t_star<1)]
t = np.exp(sigma**2 + np.log(mode) + sigma*t_star);
t_star = np.log(t)
plt.figure(figsize=(10,5))
plt.subplot(211)
plt.hist(t,bins=100);
plt.subplot(212)
plt.hist(t_star,bins=50,color='black',alpha=0.4);
print(np.mean(t_star) - sigma**2 - np.log(mode))
plt.tight_layout()


In [ ]:
np.exp(1 + np.log(0.1))

### Log normal distribution

In [ ]:
np.exp(1 + np.log(0.1))

In [ ]:
t = np.random.normal(0,1,10)
print(t)
idxs = np.logical_and(t>-1, t<1) 
print(idxs)
print(t[idxs])

### Rescaled distribution

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def norm(x,mu,sigma):
    return (1/(sigma*np.sqrt(2*np.pi)))*np.exp(-((x-mu)/sigma)**2)
    
x = np.r_[-10:10:0.1]
fx = norm(x,0,1)
alpha = 10
x_star = np.r_[-alpha*10:alpha*10:0.1]
fx_star = norm(x_star/alpha,0,1)*(1/alpha)
plt.figure(figsize=(5,5))
plt.plot(x,fx)
plt.figure(figsize=(5,5))
plt.plot(x_star,fx_star)


### Plot preprocessed retrospective data

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import lib.io.stan
import glob
import os

In [ ]:
t = []
for fname in glob.glob('results/exp10/exp10.65/id004_bj/Rfiles/fit_data*.R'):
    print(fname)
    t.append(lib.io.stan.rload(fname))

In [ ]:
plt.figure(figsize=(25,5))
plt.plot(t[3]['slp'], color='black', alpha=0.3);

In [ ]:
plt.figure(figsize=(25,25))
for i,szr_data in enumerate(t):
    plt.subplot(4,1,i+1)
    plt.plot(szr_data['slp'], color='black', alpha=0.3)

### pymc3 test

In [ ]:
%matplotlib inline
import pymc3 as pm
import numpy as np
import matplotlib.pyplot as plt
import theano.tensor as T

In [ ]:
def logNormal(x, mode=0, sd=1):
    return np.exp(sd**2 + np.log(mode) + sd*x)

In [ ]:
t = logNormal(np.random.normal(loc=0, scale=1, size=1000), mode=10, sd=0.1)
plt.hist(t)

In [ ]:
mode = 10
sd = 0.1
print('mu =', sd**2 + np.log(mode))
class mymodel:
    def __init__(self):
        self.model = pm.Model()
        with self.model as model:
            mu = pm.Uniform('mu', lower=-10, upper=10)
            t = np.ones(10).sum()
            obs = pm.Lognormal('obs', mu=mu, sd=1.0, observed=logNormal(np.random.normal(loc=0, scale=1, size=1000), mode=10, sd=0.1))
    def sample(self, iters):
        with self.model as model:
            return pm.sample(iters)
        
mdl = mymodel()
trace = mdl.sample(1000)

In [ ]:
pm.traceplot(trace)

In [ ]:
pm.plot_posterior(trace)

In [ ]:
with pm.Model() as model:
    t = pm.Normal('t', mu=0, sd=0.1)
    print(np.exp(t.logp({'t':0})))

In [ ]:
with pm.Model() as model:
    t1 = pm.Normal('t1', mu=0, sd=0.1, shape=(3,3))
    print(np.exp(t1.logp({'t1':np.zeros([3,3])})))

In [ ]:
3.989422804014327**9

In [ ]:
np.repeat(np.r_[1:10][:,np.newaxis],5,axis=1)

### Test epileptor simulation code

In [ ]:
%matplotlib inline
import numpy as np
from dyn_model import *
import matplotlib.pyplot as plt

In [ ]:
network = np.load(f'datasets/id002_cj/CJ_network.npz')
params = dict()
params['SC'] = network['SC']
params['SC'] = params['SC'] / params['SC'].max()
params['SC'][np.diag_indices(params['SC'].shape[0])] = 0
gain = network['gain_mat']



params['nn'] = params['SC'].shape[0]
params['ns'] = gain.shape[0]
params['nt'] = 150
params['I1'] = 3.1
params['time_step'] = 0.1
params['x_init'] = -2.0*np.ones(params['nn'])
params['z_init'] = 3.5*np.ones(params['nn'])
epsilon_slp = 0.1
epsilon_snsr_pwr = 5.0

params['x0'] = -2.5*np.ones(params['nn'])
params['x0'][[6,34]] = -1.8
amplitude_slp = 1.0
offset_slp = 0.0
params['tau0'] = 10
params['K'] = 1

In [ ]:
ep = Epileptor_2D()
sim_out = ep.sim(params)

In [ ]:
def dx(x, z, I1):
    dx_eval = 1 - x**3 - 2 * x**2 - z + 3.1
    return dx_eval


def dz(x, z, SC, K, x0, tau0):
    nn = x.size
    x_diff = np.repeat(x[:, np.newaxis], nn, axis=1) - x
    gx = K * SC * x_diff.T
    dz_eval = (1 / tau0) * (4 * (x - x0) - z - gx.sum(axis=1))
    return dz_eval


def step(x_prev, z_prev, dt, SC, K, x0, I1, tau0):
    x_next = x_prev + dt*dx(x_prev, z_prev, I1)
    z_next = z_prev + dt*dz(x_prev, z_prev, SC, K, x0, tau0)
    return x_next, z_next

In [ ]:
sim_out = dict()
sim_out['x'] = np.zeros([params['nt'],params['nn']])
sim_out['z'] = np.zeros([params['nt'],params['nn']])
for i in range(params['nt']):
    if(i == 0):
        sim_out['x'][i], sim_out['z'][i] = step(params['x_init'], params['z_init'], params['time_step'], params['SC'], params['K'], params['x0'], params['I1'], params['tau0'])
    else:
        sim_out['x'][i], sim_out['z'][i] = step(sim_out['x'][i-1], sim_out['z'][i-1], params['time_step'], params['SC'], params['K'], params['x0'], params['I1'], params['tau0'])

In [ ]:
sim_out = ep.sim(nt)

In [ ]:
plt.figure(figsize=(25,5))
for i in range(params['nn']):
    if i in [6,34]:
        plt.plot(sim_out['x'][:,i], 'r');
    else:
        plt.plot(sim_out['x'][:,i], 'k')
plt.figure(figsize=(25,5))
for i in range(params['nn']):
    if i in [6,34]:
        plt.plot(sim_out['z'][:,i], 'r');
    else:
        plt.plot(sim_out['z'][:,i], 'k')


In [ ]:
x[0].size

In [ ]:
np.repeat(t, 5, axis=0)

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
x = np.r_[-10:10:0.1]
y1 = -x + 4
y2 = 2*x + 5

In [ ]:
plt.figure()
plt.plot(x,y1)
plt.plot(x,y2)

In [ ]:
t = np.arange(1,17).reshape(4,4)

In [ ]:
t.max()

In [ ]:
x = np.arange(1,17).reshape(4,4)
print(x**2)


In [ ]:
(x**2).sum(axis=1)

In [ ]:
x[np.newaxis,:].repeat(4,axis=0) - x[:,np.newaxis]

In [ ]:
x[:,np.newaxis]

### Theano ODE solver

In [ ]:
%matplotlib inline
import theano
import matplotlib.pyplot as plt

In [ ]:
def y_t(y_t_prev, dt, K):
    return y_t_prev + dt*(-1*y_t_prev) + K

y_init = theano.tensor.dscalar('y_init')
dt = theano.tensor.dscalar('dt')
K = theano.tensor.dscalar('K')

output, updates = theano.scan(fn=y_t,
                              outputs_info=[y_init],
                              non_sequences=[dt, K],
                              n_steps=100)

f = theano.function(inputs=[y_init, dt, K],
                    outputs=output,
                    updates=updates,
                    on_unused_input='warn')

# y_init_val = 100
# dt_val = 0.1
# K_val = 10
# y = f(y_init_val, dt_val)

In [ ]:
y_init_val = 100
dt_val = 0.1
y = f(y_init_val, dt_val)

In [ ]:
plt.plot(y)

### Theano epileptor ODE solver

In [ ]:
%matplotlib inline
import theano
import matplotlib.pyplot as plt
import numpy as np
from dyn_model import Epileptor_2D

In [ ]:

# def dx(x, z, I1):
#     dx_eval = 1 - x**3 - 2 * x**2 - z + 3.1
#     return dx_eval

# def dz(x, z, SC, K, x0, tau0, nn):
#     x_diff = np.repeat(x[:, np.newaxis], nn, axis=1) - x
#     gx = K * SC * x_diff.T
#     dz_eval = (1 / tau0) * (4 * (x - x0) - z - gx.sum(axis=1))
#     return dz_eval

# def step(x_prev, z_prev, dt, SC, K, x0, I1, tau0, nn):
#     x_next = x_prev + dt*dx(x_prev, z_prev, I1)
#     z_next = z_prev + dt*dz(x_prev, z_prev, SC, K, x0, tau0, nn)
#     return x_next, z_next

    
# dt = theano.tensor.dscalar('dt')
# x_init = theano.tensor.dvector('x_init')
# z_init = theano.tensor.dvector('z_init')
# SC = theano.tensor.dmatrix('SC')
# I1 = theano.tensor.dscalar('I1')
# K = theano.tensor.dscalar('K')
# x0 = theano.tensor.dvector('x0')
# tau0 = theano.tensor.dscalar('tau0')
# nn = theano.tensor.iscalar('nn')

# output, updates = theano.scan(fn=step,
#                               outputs_info=[x_init, z_init],
#                               non_sequences=[dt, SC, K, x0, I1, tau0, nn],
#                               n_steps=150)

# x = output[0]
# z = output[1]

# f = theano.function(inputs=[x_init, z_init, dt, SC, K, x0, I1, tau0, nn],
#                     outputs=[x, z],
#                     updates=updates)

class epileptor2d:
    def __init__(self, nt):
        self.dt = theano.tensor.dscalar('dt')
        self.x_init = theano.tensor.dvector('x_init')
        self.z_init = theano.tensor.dvector('z_init')
        self.SC = theano.tensor.dmatrix('SC')
        self.I1 = theano.tensor.dscalar('I1')
        self.K = theano.tensor.dscalar('K')
        self.x0 = theano.tensor.dvector('x0')
        self.tau0 = theano.tensor.dscalar('tau0')
        self.nn = theano.tensor.iscalar('nn')
        self.time_step = theano.tensor.dscalar('time_step')
        self.nt = nt
        self.output, self.updates = theano.scan(fn=self.step,
                                      outputs_info=[self.x_init, self.z_init],
                                      non_sequences=[self.dt, self.SC, self.K, self.x0, self.I1, self.tau0, self.nn],
                                      n_steps=self.nt)

        self.x = self.output[0]
        self.z = self.output[1]

        self.f = theano.function(inputs=[self.x_init, self.z_init, self.dt, self.SC, self.K, self.x0, self.I1, self.tau0, self.nn],
                            outputs=[self.x],
                            updates=self.updates)

    def dx(self, x, z, I1):
        dx_eval = 1 - x**3 - 2 * x**2 - z + 3.1
        return dx_eval

    def dz(self, x, z, SC, K, x0, tau0, nn):
        x_diff = np.repeat(x[:, np.newaxis], nn, axis=1) - x
        gx = K * SC * x_diff
        dz_eval = (1 / tau0) * (4 * (x - x0) - z - gx.sum(axis=1))
        return dz_eval

    def step(self, x_prev, z_prev, dt, SC, K, x0, I1, tau0, nn):
        x_next = x_prev + dt * self.dx(x_prev, z_prev, I1)
        z_next = z_prev + dt * self.dz(x_prev, z_prev, SC, K, x0, tau0, nn)
        return x_next, z_next
    
    def sim(self, x_init_val, z_init_val, dt_val, SC_val, K_val, x0_val, I1_val, tau0_val, nn_val):
        return self.f(x_init_val, z_init_val, dt_val, SC_val, K_val, x0_val, I1_val, tau0_val, nn_val)
        

In [ ]:
network = np.load(f'datasets/id002_cj/CJ_network.npz')
SC_val = network['SC']
SC_val = SC_val / SC_val.max()
SC_val[np.diag_indices(SC_val.shape[0])] = 0
# gain_mat = network['gain_mat']



nn_val = SC_val.shape[0]
# ns = gain_mat.shape[0]
nt_val = 150
I1_val = 3.1
dt_val = 0.1
x_init_val = -2.0*np.ones(nn_val)
z_init_val = 3.5*np.ones(nn_val)
K_val = 1
time_step_val = 0.1
# epsilon_slp = 0.1
# epsilon_snsr_pwr = 5.0
# sigma=0.01

x0_val = -2.5*np.ones(nn_val)
x0_val[[6,34,58]] = -1.8
# amplitude_slp = 1.0
# offset_slp = 0.0
tau0_val = 10
# ep = epileptor2d(nt_val)
ep = Epileptor_2D(nt_val)
xt,zt = ep.sim(x_init_val, z_init_val, dt_val, SC_val, K_val, x0_val, I1_val, tau0_val, nn_val)

In [ ]:
xt.shape

In [ ]:
plt.figure()
plt.subplot(131)
plt.plot(xt[:,6])
plt.subplot(132)
plt.plot(xt[:,34])
plt.subplot(133)
plt.plot(xt[:,58])

In [ ]:
# plt.figure(figsize=(25,5))
# plt.plot(xt);
plt.figure(figsize=(25,5))
for i in range(nn_val):
    if i in [6,34]:
        plt.plot(xt[:,i], 'r');
    else:
        plt.plot(xt[:,i], 'k')
plt.figure(figsize=(25,10))
for i in range(nn_val):
    if i in [6,34]:
        plt.plot(zt[:,i], 'r');
    else:
        plt.plot(zt[:,i], 'k')

In [ ]:
import numpy as np

In [ ]:
t = np.ones([10,15])

In [ ]:
t[0].size

In [ ]:
import theano

In [ ]:
t1 = theano.tensor.dvector('t1')
t2 = theano.tensor.repeat(t1, 5, axis=1)

t3 = t2 + 10

f = theano.function()

In [ ]:
t = dict()

## PyMC3 transformed variables

In [ ]:
import theano
import pymc3 as pm
import numpy as np
import matplotlib.pyplot as plt
import lib.pymc3.transforms as tr

In [ ]:
with pm.Model() as mdl:
    alpha = pm.Uniform('alpha', lower=0)
    x = pm.Normal('x', mu=10, sd=1, transform=tr.linear(alpha, 0))
    approx = pm.fit()
    trace = approx.sample()

In [ ]:
plt.figure(figsize=[12,5])
plt.subplot(121)
plt.hist(trace['x'])
plt.subplot(122)
plt.hist(trace['x_linear__'])

In [ ]:
class scaled(pm.distributions.transforms.ElemwiseTransform):
    name = "scaled"
    def __init__(self, alpha):
        super(pm.distributions.transforms.ElemwiseTransform,self).__init__()
        self.alpha = alpha
    def forward(self, x):
        return self.alpha*x
    def backward(self, x):
        return (1/self.alpha)*x
    def jacobian_det(self, x):
        return -theano.tensor.log(self.alpha)
with pm.Model() as mdl2:
    alpha = pm.Uniform('alpha',lower=0)
    x = pm.Normal('x', mu=10, sd=1.0, transform=scaled(alpha))
    approx = pm.fit()

In [ ]:
trace = approx.sample(1000)

In [ ]:
plt.figure()
plt.hist(trace['x'],color='black',alpha=0.3)
plt.figure()
plt.hist(trace['x_scaled__'],color='blue',alpha=0.1)

In [ ]:
plt.hist(trace['alpha'])

In [ ]:
import pymc3 as pm

In [ ]:
trace = pm.backends.load_trace('results/exp10/exp10.66')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
t = np.load('results/exp10/exp10.65.1/id004_bj/samples/0/samples.npz')
ez_hyp = np.where(np.loadtxt('datasets/RetrospectivePatients/id004_bj/tvb/ez_hypothesis.destrieux.txt') == 1)[0] + 1
# ez_hyp = np.loadtxt('datasets/RetrospectivePatients/id004_bj/tvb/ez_hypothesis.destrieux.txt')

In [ ]:
ez_hyp = np.where(np.loadtxt('datasets/RetrospectivePatients/id004_bj/tvb/ez_hypothesis.destrieux.txt') == 1)
print(ez_hyp)

In [ ]:
plt.figure(figsize=(25,5))




In [ ]:
import lib.io.stan
import lib.plots.stan
import numpy as np
%matplotlib inline

In [ ]:
fname_suffix = 'hyper_fixed_md15_delta0.95'
results_dir = 'results/exp10/exp10.57.7'

syn_data = np.load('datasets/id002_cj/CJ_syn_tvb_ez=6-34_pz=27-5-11.npz')
fit_data = lib.io.stan.rload(f'{results_dir}/Rfiles/fit_data_snsrfit_ode_{fname_suffix}.R')
x0_true = syn_data['x0']
ez = syn_data['ez']
pz= np.concatenate((syn_data['pz_x0'], syn_data['pz_kplng']))
# szng_roi = np.setxor1d(ez, pz)
# non_szng_roi = np.setdiff1d(np.r_[0:nn], szng_roi)

variables_of_interest = ['lp__','accept_stat__','stepsize__','treedepth__','n_leapfrog__',\
                         'divergent__', 'energy__','x0',  'x', 'z', 'mu_slp', 'mu_snsr_pwr',\
                         'amplitude', 'offset', 'tau0', 'K', 'alpha']
chains = [36]
nwarmup = 0
nsampling = 2000
params = ['tau0', 'amplitude', 'offset', 'K', 'alpha']
for chain_no in chains:
    pstr_samples = lib.io.stan.read_samples([f'{results_dir}/samples_{fname_suffix}_chain{chain_no}.csv'],
                                      variables_of_interest=variables_of_interest, nwarmup=nwarmup, nsampling=nsampling)
#     for var in pstr_samples.keys():
#        pstr_samples[var] = pstr_samples[var][200:]
    lib.plots.stan.x0_violin_syn(pstr_samples['x0'], x0_true, ez, pz, figsize=(25,5),
                                 legend_loc='lower right')
    lib.plots.stan.nuts_diagnostics(pstr_samples, figsize=(15,10))
    lib.plots.stan.pair_plots(samples=pstr_samples, params=params,
                              sampler='HMC')
    
    x_pp_mean = np.mean(pstr_samples['x'], axis=0)
    z_pp_mean = np.mean(pstr_samples['z'], axis=0)
    lib.plots.stan.plot_source(x_pp_mean, z_pp_mean, ez, pz)
#     lib.plots.stan.plot_phase(x_pp_mean, z_pp_mean, ez, pz, non_szng_roi)

    slp = pstr_samples['mu_slp'].mean(axis=0)
    snsr_pwr = pstr_samples['mu_snsr_pwr'].mean(axis=0)
    lib.plots.stan.plot_fit_target(data_pred={'slp':slp,'snsr_pwr':snsr_pwr}, data_true={'slp':fit_data['slp'],'snsr_pwr':fit_data['snsr_pwr']})

In [ ]:
import os

In [ ]:
os.path.dirname(os.getcwd())

In [ ]:
%matplotlib inline
import numpy as np
import lib.io.stan
import lib.plots.stan
import lib.utils.stan
import subprocess
import matplotlib.pyplot as plt
import os 
from matplotlib.lines import Line2D
import importlib

In [ ]:
data_dir = 'datasets/id002_cj'
results_dir = 'results/exp10/exp10.57.18'
os.makedirs(results_dir,exist_ok=True)
os.makedirs(f'{results_dir}/logs',exist_ok=True)
os.makedirs(f'{results_dir}/figures',exist_ok=True)

network = np.load(f'{data_dir}/CJ_network.npz')
SC = network['SC']
K = np.max(SC)
print(K)

### Retro preprocessing

In [ ]:
%matplotlib inline
import numpy as np
import lib.io.stan
import lib.plots.stan
import matplotlib.pyplot as plt
import os
from matplotlib.lines import Line2D
import retro_prepare_data
import matplotlib.colors

In [ ]:
data_dir = 'datasets/retro/id023_br'
szr_name = 'BR_crise1_PSG_130312B-DEX_0006'
meta_data_fname = f'{szr_name}.json'
raw_seeg_fname = f'{szr_name}.raw.fif'
# fname_suffix = f'{szr_name}'
# if os.path.isdir(results_dir):
#     os.rmdir(results_dir)
# os.makedirs(results_dir,exist_ok=True)
# os.makedirs(f'{results_dir}/logs',exist_ok=True)
# os.makedirs(f'{results_dir}/figures',exist_ok=True)
# os.makedirs(f'{results_dir}/Rfiles', exist_ok=True)

In [ ]:
hpf = 10
lpf = 0.02
data = retro_prepare_data.prepare_data(data_dir, meta_data_fname, raw_seeg_fname, hpf, lpf)
# fname_suffix += f'_hpf{hpf}_lpf{lpf}'

In [ ]:
ds_freq = int(data['slp'].shape[0]/150)
data['slp'] = data['slp'][0:-1:ds_freq]
# data['snsr_pwr'] = (data['slp']**2).mean(axis=0)
data['ns'], data['nn'] = data['gain'].shape
# data['nt'] = data['slp'].shape[0]
# ez_hyp = np.where(np.loadtxt(f'{data_dir}/tvb/ez_hypothesis.destrieux.txt') == 1)[0]
# data['x0_mu'] = -3.0*np.ones(data['nn'])
# data['x0_mu'][ez_hyp] = -1.5

In [ ]:
# plt.figure(figsize=(20,5))
# plt.subplot(121)
# plt.imshow(data['SC'],norm=matplotlib.colors.LogNorm(vmin=1e-6, vmax=data['SC'].max()));
# plt.colorbar(fraction=0.046,pad=0.04);
# plt.title('Normalized SC (log scale)',fontsize=12, fontweight='bold')

# plt.subplot(122)
# plt.imshow(data['gain'],norm=matplotlib.colors.LogNorm(vmin=data['gain'].min(), vmax=data['gain'].max()));
# plt.colorbar(fraction=0.046,pad=0.04);
# plt.xlabel('Region#', fontsize=12)
# plt.ylabel('Channel#', fontsize=12)
# plt.title('Gain Matrix (log scale)',fontsize=12, fontweight='bold')
# # plt.savefig(f'{results_dir}/figures/network.png')

plt.figure(figsize=(25,13))
plt.subplot(211)
plt.plot(data['slp'], color='black', alpha=0.3);
plt.xlabel('Time', fontsize=12)
plt.ylabel('SLP', fontsize=12)

plt.subplot(212)
plt.bar(np.r_[1:data['ns']+1],data['snsr_pwr'], color='black', alpha=0.3);
plt.xlabel('Time', fontsize=12)
plt.ylabel('Power', fontsize=12)
plt.title('SEEG channel power', fontweight='bold')
# plt.savefig(f'{results_dir}/figures/fitting_target_{fname_suffix}.png')
# plt.tight_layout()


### Find EZ by using seizure onsets

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import lib.io.stan
import lib.plots.stan
import dyn_model

In [ ]:
fit_data = lib.io.stan.rload('results/exp10/exp10.67/id002_sd/Rfiles/fit_data_SDcrise1_0002_hpf10_lpf0.05.R')
optima = lib.io.stan.read_samples(['results/exp10/exp10.67/id002_sd/samples_SDcrise1_0002_hpf10_lpf0.05_chain1.csv'])
ez_hyp = np.where(np.loadtxt('datasets/retro/id002_sd/tvb/ez_hypothesis.destrieux.txt') == 1)[0]

In [ ]:
x = optima['x'][0]

In [ ]:
def find_ez(x, onst_thrshld):
    nn = x.shape[1]
    onsets = 200*np.ones(nn)
    for i in range(nn):
        xt = x[:,i] > onst_thrshld
        if(xt.any()):
            onsets[i] = np.where(x[:,i] > onst_thrshld)[0][0]
    a, b = np.histogram(onsets[onsets<150])
    ez_pred = np.zeros(nn)
    ez_pred[np.where(onsets<b[2])] = 1
#     np.save(os.path.join(root_dir, patient_id, 'ez_pred.npy'), ez_pred)
    return onsets,ez_pred

In [ ]:
onsets,ez_pred = find_ez(x, -0.05)
onsets = onsets[onsets<150]

In [ ]:
ez_pred

In [ ]:
np.where(ez_pred!=0)

In [ ]:
plt.hist(onsets)

In [ ]:
ep_params = dict()
ep_params['nt'] = fit_data['nt']
ep_params['nn'] = fit_data['nn']
ep_params['time_step'] = 0.1
ep_params['I1'] = 3.1
ep_params['tau0'] = optima['tau0'][0]
ep_params['x_init'] = optima['x_init'][0]
ep_params['z_init'] = optima['z_init'][0]
ep_params['x0'] = optima['x0'][0]
ep_params['SC'] = fit_data['SC']
ep_params['K'] = 0 #optima['K'][0]
eplptr = dyn_model.Epileptor_2D()
x,z = eplptr.sim(ep_params)

In [ ]:
lib.plots.stan.plot_source(optima['x'][0], optima['z'][0], ez_hyp, [])

In [ ]:
lib.plots.stan.plot_source(x, z, ez_hyp, [])

In [ ]:
for i in range(ep_params['nt']):
    xt = x[i,:] > -0.25
    if(any(xt)):
        print(np.where(xt))
        break

In [ ]:
np.linspace(-0.25, 0.0, int(0.25/0.01)+1)

In [ ]:
import glob

In [ ]:
glob.glob('results/exp10/exp10.67/id001_bt/*chain1.csv')

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
ax = plt.subplot(111)
ax.set_yticks(ax.get_yticks())
ax.set_yticklabels(ax.get_yticklabels())

## Plot electrodes

In [ ]:
import sys
sys.path.append('tvbpipeline')
import util.plot
import os

In [ ]:
data_dir = 'datasets/syn_data/id001_bt'
figs_dir = 'datasets/syn_data/id001_bt/figures'
os.makedirs(figs_dir, exist_ok=True)
util.plot.seeg_elecs(os.path.join(data_dir, 'connectivity.destrieux.zip'), 
                     os.path.join(data_dir, 'seeg.xyz'), ez_idx=[], pz_idx=[], 
                     out_fig=os.path.join(figs_dir, 'electrodes.svg'))

## Find EZ and PZ based on onset

In [1]:
import retro_results
import numpy as np
import matplotlib.pyplot as plt

In [2]:
ez, pz, onsets = retro_results.find_ez_single(0, 1, 10, 'results/exp10/exp10.72/samples_syn_optim_chain1.csv')

In [3]:
ez, pz, onsets

((array([34, 79]),),
 (array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
          13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
          26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
          39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
          52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
          65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
          78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
          91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
         104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
         117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
         130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
         143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
         156, 157, 158, 159, 160, 161, 162, 163]),),
 array([200., 200., 200., 200., 200., 

In [ ]:
t

In [ ]:
np.nonzero(np.logical_and(t > 4, t < 13))